In [1]:
import tensorflow_decision_forests as tfdf
#import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

2024-05-13 11:00:02.195808: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 11:00:02.267272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 11:00:02.267338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 11:00:02.269110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 11:00:02.279965: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 11:00:02.280994: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
len(tf.config.list_physical_devices('GPU'))

2024-05-13 11:00:07.578470: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 11:00:07.579154: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


0

In [2]:
INPUT_FILE_PATH = "../data/5_lichess_csv_ML_ready/Elo_1500_1600.csv.pkl"

In [3]:
df = pd.read_pickle(INPUT_FILE_PATH)

#! remove draws for now
df = df[df['Result'] != 2]

#! remove unwanted columns in the training
df = df.drop(columns=['Board','MoveNumber','WhiteElo','BlackElo'])
df = df.reset_index(drop=True)
df = df.reindex(sorted(df.columns), axis=1)

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="Result")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="Result")


2024-04-17 10:55:19.711945: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-17 10:55:19.712337: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# Create a Random Search tuner with 10 trials and automatic hp configuration.
tuner = tfdf.tuner.RandomSearch(num_trials=10, use_predefined_hps=True)

In [6]:
# Train the model
model = tfdf.keras.RandomForestModel(tuner=tuner)
model.fit(train_ds)

Use /tmp/tmp5oq85yt1 as temporary training directory


Reading training dataset...
Training dataset read in 0:00:11.291876. Found 508788 examples.
Training model...


In [ ]:
# Evaluate the model
model.compile(["accuracy"])
tuned_test_accuracy = model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy with the TF-DF hyper-parameter tuner: {tuned_test_accuracy:.4f}")